In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# !pip install dask-ml[complete]
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import dask.dataframe as dd
import seaborn as sns 
import dask_ml
import matplotlib.pyplot as plt
import gc

In [3]:
import numpy as np
import plotly.express as px
import dask.array as da
import pandas as pd

In [4]:
columns = list(dd.read_csv("/kaggle/input/nanoutlierremoveddataset/NaNOutlierRemovedDataset.csv",dtype={'Census_ProcessorClass':'object','PuaMode':'object'}).head())
train_dataset = dd.read_csv("/kaggle/input/nanoutlierremoveddataset/NaNOutlierRemovedDataset.csv",dtype={'Census_ProcessorClass':'object','PuaMode':'object'},usecols = [i for i in columns if i != 'PuaMode'])


Next cell is to show the distribution of target values in the columns to label encode to show the ordering.

In [5]:
train_df = pd.read_csv("/kaggle/input/nanoutlierremoveddataset/NaNOutlierRemovedDataset.csv",usecols = columns)
rows = train_df.shape[0]
label_columns = ['ProductName','EngineVersion','AppVersion','AvSigVersion','AVProductsInstalled','AVProductsEnabled']

d_list =[]
for c in label_columns:
#     print(train_df[c].unique())
    values = list(train_df[c].unique())
#     print(values)
    yes,no,labels = [],[],[]
    total = 0
    dic = {}
    for val in values:
#         print(train_dataset[da.logical_and(train_dataset[c]==values[key],train_dataset['HasDetections']==1)].compute().shape[0])
#         y = int(train_df[(train_df[c]==val and train_df['HasDetections']==1)].shape[0])
        temp = train_df[train_df[c]==val]
        y = int(temp[temp['HasDetections']==1].shape[0])
        n = int(temp[temp['HasDetections']==0].shape[0])
        del temp
        gc.collect()
        yes.append(y)
        no.append(n)
        dic[val]=y+n
        labels.append(val)
    #sort dic
    d = dict(sorted(dic.items(), key = lambda kv: kv[1],reverse=True))
    print(d)
    d_list.append(d)
    fig,ax = plt.subplots()
    ax.bar(labels,yes,0.35,label = 'Yes')
    ax.bar(labels,no,0.35,label = 'No')
    ax.set_ylabel("Total number of values")
    ax.set_title("Distribution of output values in "+c)
    ax.legend()
    plt.show()


In [ ]:
columns_to_drop = ['AvSigVersion','Census_OSVersion','OsBuildLab']
for c in columns_to_drop:
    train_dataset = train_dataset.drop(c,axis=1)

In [ ]:
columns_to_get_from_test = train_dataset.columns.tolist()
columns_to_get_from_test.remove('HasDetections')
categorical_columns = []
for c in train_dataset.columns:
    if train_dataset[c].dtype == 'object':
        categorical_columns.append(c)

train_dataset = train_dataset.drop(columns ='MachineIdentifier')
train_dataset = train_dataset.categorize()

In [ ]:
continuous_columns = train_dataset.columns.tolist()
for c in categorical_columns:
    if c in continuous_columns:
        continuous_columns.remove(c)
categorical_columns.remove('MachineIdentifier')
train_dataset = dd.get_dummies(train_dataset,columns = categorical_columns)

y_train = train_dataset['HasDetections']
train_dataset = train_dataset.drop(columns='HasDetections')

In [ ]:
test_dataset = pd.read_csv("../input/save-the-attack-contest/test_data.csv", usecols = [i for i in columns_to_get_from_test])

test_machine_id = test_dataset['MachineIdentifier']
test_dataset = test_dataset.drop(columns = 'MachineIdentifier')

for column in categorical_columns:
    test_dataset[column].fillna(test_dataset[column].value_counts().idxmax(), inplace=True)

In [ ]:
for column in continuous_columns:
    if(column == 'HasDetections'):
        continue
    test_dataset[column].fillna(test_dataset[column].mean(), inplace=True)

In [ ]:
test_dataset.to_csv('test_dataset_filled.csv',index = False)

In [ ]:
test_dataset = dd.read_csv("./test_dataset_filled.csv")

#one hot encoding
categorical_columns = test_dataset.select_dtypes(include=['object']).columns.tolist()
test_dataset = test_dataset.categorize()
test_dataset = dd.get_dummies(test_dataset,columns = categorical_columns)

In [ ]:
train_columns = train_dataset.columns.tolist()
test_columns = test_dataset.columns.tolist()

for column in train_columns:
    if column not in test_columns:
        train_dataset = train_dataset.drop(columns = column)

train_columns = train_dataset.columns.tolist()
test_columns = test_dataset.columns.tolist()

for column in test_columns:
    if column not in train_columns:
        test_dataset = test_dataset.drop(columns = column)

In [ ]:
train_dataset = train_dataset[sorted(train_dataset.columns)]
test_dataset = test_dataset[sorted(test_dataset.columns)]

In [ ]:
from dask_ml.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_dataset = scaler.fit_transform(train_dataset)
test_dataset = scaler.transform(test_dataset)

In [ ]:
#PCA
from dask_ml.decomposition import IncrementalPCA

pca = IncrementalPCA(n_components = 7,copy=False)

In [ ]:
pca.fit(train_dataset.to_dask_array(lengths=True))

In [ ]:
pca.transform(test_dataset.to_dask_array(lengths=True))

In [ ]:
# train_dataset.to_csv("trainDatasetFinal.csv",index=False,single_file=True)
# test_dataset.to_csv("test_dataset_final.csv",index=False,single_file=True)

In [ ]:
# sgdClassifier
from dask_ml.datasets import make_classification

from dask_ml.wrappers import Incremental

from sklearn.linear_model import SGDClassifier

estimator = SGDClassifier(loss='log', max_iter=1000)

clf = Incremental(estimator)

clf.fit(train_dataset, y_train, classes = [0, 1])

import dask.bag as db

ypred = clf.predict(test_dataset)
ypred = ypred.compute()
prediction_df = pd.DataFrame({'MachineIdentifier':test_machine_id,'HasDetections':ypred})
prediction_df.to_csv('test_data_predictions1.csv',index = False)

In [ ]:
# MLP classifier
from dask_ml.datasets import make_classification

from dask_ml.wrappers import Incremental

from sklearn.neural_network import MLPClassifier

estimator = MLPClassifier(activation='logistic', max_iter=3000)

clf = Incremental(estimator)

clf.fit(train_dataset, y_train, classes = [0, 1])

import dask.bag as db

ypred = clf.predict(test_dataset)
ypred = ypred.compute()

prediction_df = pd.DataFrame({'MachineIdentifier':test_machine_id,'HasDetections':ypred})
prediction_df.to_csv('test_data_predictions1.csv',index = False)

In [ ]:
#xgboost

In [ ]:
#adaboost